# SIA Service Test Notebook

Automated tests for the Simple Image Access (SIA) v2 service.

## Imports

In [ ]:
from lsst.rsp import get_siav2_service, get_datalink_result
from astropy.time import Time

## Service Instantiation

In [ ]:
service = get_siav2_service("dp1")
assert service is not None
print(f"SIA service URL: {service.baseurl}")

## Cone Search - Small Radius

In [ ]:
target_ra = 62.0
target_dec = -37.0
radius = 0.01

results = service.search(pos=(target_ra, target_dec, radius), maxrec=10)
print(f"Retrieved {len(results)} images")
assert len(results) >= 0
if len(results) > 0:
    results.to_table()[:5]

## Cone Search - Larger Radius

In [ ]:
radius = 0.1

results = service.search(pos=(target_ra, target_dec, radius), maxrec=1000)
print(f"Retrieved {len(results)} images")
assert len(results) >= 0

## Search by Calibration Level

In [ ]:
results = service.search(
    pos=(target_ra, target_dec, 0.05),
    calib_level=2,
    maxrec=20
)
print(f"Retrieved {len(results)} visit images (calib_level=2)")
assert len(results) >= 0
if len(results) > 0:
    results.to_table()[:5]

## Search by Data Product Subtype

In [ ]:
results = service.search(
    pos=(target_ra, target_dec, 0.1),
    calib_level=3,
    dpsubtype='lsst.deepCoadd_calexp',
    maxrec=20
)
print(f"Retrieved {len(results)} deepCoadd_calexp images")
assert len(results) >= 0

## Search by Band/Filter (r-band)

In [ ]:
r_band = (5.520e-07, 6.910e-07)

results = service.search(
    pos=(target_ra, target_dec, 0.05),
    band=r_band,
    calib_level=2,
    maxrec=20
)
print(f"Retrieved {len(results)} r-band images")
assert len(results) >= 0

## Datalink Access from SIA Result

In [ ]:
results = service.search(
    pos=(target_ra, target_dec, 0.05),
    calib_level=3,
    maxrec=5
)
print(f"Retrieved {len(results)} images for datalink test")
assert len(results) > 0, "Need at least one image for datalink test"

## Datalink - Get Datalink Result

In [ ]:
result = results[0]
dl_result = get_datalink_result(result)
print(f"Datalink result has {len(dl_result)} entries")
assert len(dl_result) > 0
dl_result.to_table()

## Datalink - Extract Access URL

In [ ]:
access_url = dl_result.getrecord(0).get('access_url')
print(f"Access URL: {access_url}")
assert access_url is not None

## Combined Constraints Search

In [ ]:
# r-band range
r_band = (5.520e-07, 6.910e-07)

results = service.search(
    pos=(target_ra, target_dec, 0.1),
    band=r_band,
    calib_level=2,
    maxrec=30
)
print(f"Retrieved {len(results)} r-band visit images")
assert len(results) >= 0

## Test maxrec Limit

In [ ]:
results_5 = service.search(
    pos=(target_ra, target_dec, 0.1),
    calib_level=2,
    maxrec=5
)
print(f"With maxrec=5: Retrieved {len(results_5)} images")
assert len(results_5) <= 5

## Verify Result Columns

In [ ]:
results = service.search(
    pos=(target_ra, target_dec, 0.05),
    calib_level=3,
    maxrec=5
)

if len(results) > 0:
    table = results.to_table()
    print(f"Columns in result: {table.colnames}")
    # Check for standard SIA columns
    expected_columns = ['dataproduct_type', 'calib_level', 's_ra', 's_dec', 'access_url']
    for col in expected_columns:
        if col in table.colnames:
            print(f"  Found expected column: {col}")
else:
    print("No results to check columns")

## Test Complete

In [ ]:
print("All SIA service tests completed successfully.")